In [ ]:
%pip install faiss-cpu sentence-transformers

In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores.faiss import FAISS
import os
import getpass

In [2]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter in your OpenAI API Key:")

# Loader = PyPDFLoader("data/principles_of_marketing_book.pdf") # If the file was local
loader = PyPDFLoader(
    "https://storage.googleapis.com/strapi_cms_assets/principles_of_marketing_book.pdf"
)
raw_documents = loader.load_and_split()
print(raw_documents[0])

page_content='Principles of Mark eting' metadata={'source': 'https://storage.googleapis.com/strapi_cms_assets/principles_of_marketing_book.pdf', 'page': 0}


In [3]:
len(raw_documents)

501

In [4]:
# Let's just use the first 100 documents for this example:
raw_documents = raw_documents[:100]

In [5]:
# Load the marketing principles .pdf, split it into chunks, embed each chunk and load it into the vector store.
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
# You can use OpenSource embeddings instead of OpenAIEmbeddings --> embeddings = HuggingFaceEmbeddings()
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(documents, embeddings)

In [6]:
db

In [7]:
query = "What is the license used within the principles of marketing book?"
docs = db.similarity_search(query)
print(docs[0].page_content)

Principles of Marketing by University of Minnesota is licensed under a Creative Commons Attribution-NonCommer cial-Shar eAlike 4.0
International License , except wher e otherwise noted.


# Building a simple QA retrieval system using FAISS


By combining similarity search across a vector database with a chat model, you can easily create a very simple question answering (QA) system. Expanding on the GPT best practices to avoid hallucinations by asking the chat model to only answer using reference text from the database, you can create a simple system that can answer questions about a specific topic.


In [9]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts.chat import SystemMessagePromptTemplate, ChatPromptTemplate

query = "License used within the principles of marketing book"
docs = db.similarity_search(query)

# Combine all of the docs into a single string:
combined_docs = " ".join([doc.page_content for doc in docs])

# Check the length of the combined docs:
print(len(combined_docs))

# Create the chat model:
chat = ChatOpenAI()

template = """Given the following text, answer the following question.
        Question: {question}
        
        You must follow the following principles:
        - You must only answer using the reference text provided.
        - If you don't the answer without the reference text, you must return "I don't know".
        - It is vital that you return the answer with the reference text and not without.

        Reference Text: {combined_docs}"""

# Make the template:
system_message = SystemMessagePromptTemplate.from_template(template)
chat_prompt = ChatPromptTemplate.from_messages([system_message])

# Create the messages:
query = "What is the license used within the principles of marketing book?"
messages = chat_prompt.format_prompt(
    question=query, combined_docs=combined_docs
).to_messages()

# Call the chat model:
response = chat(messages)
print(response.content)

# Alternatively if we try something that the LLM doesn't have access too within the .pdf marketing book:
query = "What is data science?"
messages = chat_prompt.format_prompt(
    question=query, combined_docs=combined_docs
).to_messages()

# Call the chat model:
response = chat.invoke(messages)
print(response.content)

1253
The license used within the Principles of Marketing book is a Creative Commons Attribution-NonCommercial-ShareAlike license.
I don't know.


---


It's possible write the above code in a simpler way with _a chain_. For now think of a chain as a _multiple steps that are executed to accomplish a task_. Additionally you will use the vector database as your back end for searching results, commonly referred to as a _retriever._


In [10]:
db.as_retriever()

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fa311290130>)

In [11]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=chat,
    chain_type="map_reduce",
    retriever=db.as_retriever(),
    return_source_documents=True,
)
qa.invoke({"query": "What is book's title?"})

{'query': "What is book's title?",
 'result': 'The provided text does not mention the title of the book.',
 'source_documents': [Document(page_content='Principles of Mark eting\n[Author remo ved atrequest oforiginal publisher]\nUNIVERSITY OF MINNESO TALIBRARIES PUBLISHING EDITION, 2015. THIS EDITION AD APTED\nFROM AWORK ORIGINALL YPRODUCED IN 2010 BY APUBLISHER WHO HAS REQUESTED THA TIT\nNO TRECEIVE ATTRIBUTION.\nMINNEAPOLIS, MN', metadata={'source': 'https://storage.googleapis.com/strapi_cms_assets/principles_of_marketing_book.pdf', 'page': 1}),
  Document(page_content='Publisher Information\nPrinciples of Marketing is adapted\nfrom a work produced and distributed\nunder a Creative Commons license\n(CC BY -NC-SA) in 2010 by a\npublisher who has requested that they\nand the original author not receive\nattribution. This adapted edition is\nproduced by the University of Minnesota Libraries Publishing through the eLearning Support Initiative.\nThis adaptation has reformatted the original

You can find more information about Document QA retrieval here https://python.langchain.com/docs/modules/chains/additional/question_answering.html.


---

# Similarity Search with score


FAISS-specific techniques exist, one of which is termed as `.similarity_search_with_score`. This particular method permits the retrieval of not only the corresponding documents but also the distance measure between the query and the said documents. The returned distance measure utilizes L2 distance, where a smaller score is a better match.


In [12]:
query = "What is the license used within the principles of marketing book?"
docs_and_scores = db.similarity_search_with_score(query=query)

In [13]:
docs_and_scores

[(Document(page_content='Principles of Marketing by University of Minnesota is licensed under a Creative Commons Attribution-NonCommer cial-Shar eAlike 4.0\nInternational License , except wher e otherwise noted.', metadata={'source': 'https://storage.googleapis.com/strapi_cms_assets/principles_of_marketing_book.pdf', 'page': 2}),
  0.22722036),
 (Document(page_content='Publisher Information\nPrinciples of Marketing is adapted\nfrom a work produced and distributed\nunder a Creative Commons license\n(CC BY -NC-SA) in 2010 by a\npublisher who has requested that they\nand the original author not receive\nattribution. This adapted edition is\nproduced by the University of Minnesota Libraries Publishing through the eLearning Support Initiative.\nThis adaptation has reformatted the original text, and replaced some images and figures to make the resulting\nwhole more shareable. This adaptation has not significantly altered or updated the original 2010 text. This work\nis made available under t

It's possible to add documents to the FAISS vector store.


In [14]:
new_documents = [
    Document(
        page_content="Data engineering begins with data collection. Often, data streams in from multiple sources, such as customer interactions, website activity, social media, IoT devices, and more.",
        metadata={"title": "Data Engineering Book"},
    ),
    Document(
        page_content="Pandas is a popular open-source Python library widely used for data manipulation and analysis. It provides powerful data structures like DataFrames and Series, which allow users to handle and analyze structured data easily. ",
        metadata={"title": "Pandas Analysis Book"},
    ),
]

In [15]:
db.add_documents(documents=new_documents)

['aa547b8b-b0b9-4d99-bf6d-ffbe76267f4d',
 'a01cc703-9d6b-4723-ba4e-defb6ced0596']

In [16]:
# Now if you search for pandas you should see the new documents in the results.
db.similarity_search_with_score(query="pandas")

[(Document(page_content='Pandas is a popular open-source Python library widely used for data manipulation and analysis. It provides powerful data structures like DataFrames and Series, which allow users to handle and analyze structured data easily. ', metadata={'title': 'Pandas Analysis Book'}),
  0.22036263),
 (Document(page_content='Figure 3.1 1\nThe hike up to Mount Everest used to be pristine. Now it looks\nmore like this. Who’ s responsible? Are consumers or companies\nresponsible, or both?\njqpubliq – Recycling Center Pile – CC BY -SA 2.0.\nOther companies are less concerned about conservation than they are about planned obsolescence . Planned\nobsolescence is a deliberate ef fort by companies to make their products obsolete, or unusable, after a period of\ntime. The goal is to improve a company’ s sales by reducing the amount of time between the repeat purchases\nconsumers make of products. When a software developer introduces a new version of product, it is usually\ndesigned to

---

# Loading and Saving the Vector Store


In [17]:
db.save_local(
    "data/vectorstore"
)  # This creates a index.faiss and index.pkl file in the data/vectorstore directory.

In [18]:
new_db = FAISS.load_local("data/vectorstore", embeddings)

docs = new_db.similarity_search_with_score(query="pandas")

print(docs[0])

(Document(page_content='Pandas is a popular open-source Python library widely used for data manipulation and analysis. It provides powerful data structures like DataFrames and Series, which allow users to handle and analyze structured data easily. ', metadata={'title': 'Pandas Analysis Book'}), 0.22036263)


In [19]:
# Merging two indexes together:
documents = [
    Document(
        page_content="I love data engineering",
        metadata={"title": "Data Engineering Book"},
    ),
    Document(
        page_content="I love pandas",
        metadata={"title": "Pandas Analysis Book"},
    ),
]


vectorstore_one = FAISS.from_documents([documents[0]], embeddings)
vectorstore_two = FAISS.from_documents([documents[1]], embeddings)

In [20]:
print(vectorstore_one.docstore._dict)
print(vectorstore_two.docstore._dict)

{'3f255782-d767-49ce-9b24-beb7d467a690': Document(page_content='I love data engineering', metadata={'title': 'Data Engineering Book'})}
{'4e52cf63-9557-4959-b424-a5a957bf4958': Document(page_content='I love pandas', metadata={'title': 'Pandas Analysis Book'})}


In [21]:
vectorstore_one.merge_from(vectorstore_two)

In [22]:
print(vectorstore_one.docstore._dict)

{'3f255782-d767-49ce-9b24-beb7d467a690': Document(page_content='I love data engineering', metadata={'title': 'Data Engineering Book'}), '4e52cf63-9557-4959-b424-a5a957bf4958': Document(page_content='I love pandas', metadata={'title': 'Pandas Analysis Book'})}


---

# Filtering and Similarity Search

The capability to filter is also available in the FAISS vectorstore. However, since FAISS does not inherently support this feature, it requires manual implementation. This process entails initially retrieving more results than `k`, followed by their filtration. Document filtration can be executed based on metadata. Additionally, you have the option to determine the quantity of documents you wish to fetch prior to filtering by setting the `fetch_k` parameter during any search method invocation. To illustrate, consider the following minor example:


In [23]:
from langchain_core.documents import Document

# Sample list of data engineering documents with content and metadata
documents = [
    Document(
        page_content="Data engineering involves designing data pipelines.",
        metadata=dict(page=1),
    ),
    Document(
        page_content="ETL is a key process in data engineering workflows.",
        metadata=dict(page=1),
    ),
    Document(
        page_content="Data modeling ensures data integrity and accessibility.",
        metadata=dict(page=2),
    ),
    Document(
        page_content="Data warehouses are centralized repositories for structured data.",
        metadata=dict(page=2),
    ),
    Document(
        page_content="Scalability and performance are crucial in data engineering.",
        metadata=dict(page=3),
    ),
    Document(
        page_content="Data lakes store raw data for future processing and analysis.",
        metadata=dict(page=3),
    ),
    Document(
        page_content="Data governance ensures data security and compliance.",
        metadata=dict(page=4),
    ),
    Document(
        page_content="Data science and data engineering collaborate for data insights.",
        metadata=dict(page=4),
    ),
]

db = FAISS.from_documents(documents=documents, embedding=embeddings)
results_with_scores = db.similarity_search_with_score(
    "data engineering", k=2, fetch_k=20, filter={"page": 1}
)  # Fetch top 20 results and return top 2 results with scores, filtered by page 1
for doc, score in results_with_scores:
    print(f"Content: {doc.page_content}, Metadata: {doc.metadata}, Score: {score}")

Content: Data engineering involves designing data pipelines., Metadata: {'page': 1}, Score: 0.13679775595664978
Content: ETL is a key process in data engineering workflows., Metadata: {'page': 1}, Score: 0.25560182332992554


---

# Maximal Marginal Relevance (MMR)

Maximal Marginal Relevance (MMR) search is an algorithm used to diversify the results obtained from an information retrieval system. It was introduced to address the issue of redundancy in information retrieval results.

In the context of generative AI models like GPT-4, an MMR search would strive to balance the trade-off between relevance (how closely the response matches the prompt) and novelty (how different each response is from the others).

The MMR algorithm works by iteratively selecting the item that maximizes a certain criterion, typically a weighted combination of relevance to the query and dissimilarity to the already selected items. The main idea is to re-rank the list of items retrieved by an initial search in order to promote diversity.

It's possible to use MMR whilst searching on a vector database:


In [24]:
results = db.max_marginal_relevance_search("data engineering", filter=dict(page=1))
for doc in results:
    print(f"Content: {doc.page_content}, Metadata: {doc.metadata}")

Content: Data engineering involves designing data pipelines., Metadata: {'page': 1}
Content: ETL is a key process in data engineering workflows., Metadata: {'page': 1}
